User
8. Create a pod that uses secrets
a. Pull secrets from environment variables
b. Pull secrets from a volume
c. Dump the secrets out via kubectl to show it worked

In [ ]:
a. Pull secrets from environment variables:

yaml
Copy code
apiVersion: v1
kind: Pod
metadata:
  name: secret-pod-env
spec:
  containers:
  - name: secret-container
    image: busybox
    command: [ "/bin/sh", "-c", "echo $SECRET_KEY && sleep 3600" ]
    env:
      - name: SECRET_KEY
        valueFrom:
          secretKeyRef:
            name: mysecret
            key: secret-key
  restartPolicy: Never
In this example, the pod reads a secret named mysecret from the environment variable SECRET_KEY.

b. Pull secrets from a volume:

yaml
Copy code
apiVersion: v1
kind: Pod
metadata:
  name: secret-pod-volume
spec:
  containers:
  - name: secret-container
    image: busybox
    command: [ "/bin/sh", "-c", "cat /etc/secret-volume/secret-file && sleep 3600" ]
    volumeMounts:
      - name: secret-volume
        mountPath: /etc/secret-volume
  volumes:
    - name: secret-volume
      secret:
        secretName: mysecret
  restartPolicy: Never
This pod mounts a secret named mysecret to a volume and then reads the secret from a file within that volume.

c. Dump the secrets out via kubectl to show it worked:

You can use the kubectl get secret command to retrieve the secrets in the cluster. Assuming you've 
created the secrets using kubectl create secret command previously:

bash
Copy code
kubectl get secret mysecret -o yaml
This command will display the YAML representation of the mysecret secret, including its data. 
You can see the values of the secrets within this output.

Remember to replace mysecret with the name of your actual secret.

9. Create a Pod with two containers, both with image busybox and command "echo hello;
sleep 3600". Connect to the second container and run 'ls'

In [ ]:
You can create a pod with two containers, both using the busybox image and executing the
command echo hello; sleep 3600. Then, you can connect to the second container and run ls.
 Below is the YAML manifest for achieving this:

yaml
Copy code
apiVersion: v1
kind: Pod
metadata:
  name: two-container-pod
spec:
  containers:
  - name: container1
    image: busybox
    command: ["/bin/sh", "-c", "echo hello; sleep 3600"]
  - name: container2
    image: busybox
    command: ["/bin/sh", "-c", "sleep 3600"]
    stdin: true
    tty: true
In this manifest:

Two containers are defined within the same pod.
Both containers use the busybox image.
Container 1 executes the command echo hello; sleep 3600.
Container 2 only sleeps for 3600 seconds.
Container 2 is configured with stdin and tty set to true, allowing us to connect to it.
To connect to the second container and run ls, you can use the kubectl exec command:

bash
Copy code
kubectl exec -it two-container-pod -c container2 -- ls
This command connects to the container2 within the two-container-pod pod and executes ls.

10. Create a pod with an nginx container exposed on port 80. Add a busybox init container
which downloads a page using "wget -O /work-dir/index.html http://neverssl.com/online".
Make a volume of type emptyDir and mount it in both containers. For the nginx container,
mount it on "/usr/share/nginx/html" and for the init container, mount it on "/work-dir".
When done, get the IP of the created pod and create a busybox pod and run "wget -O￾IP

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx-busybox-pod
spec:
  containers:
  - name: nginx-container
    image: nginx
    ports:
    - containerPort: 80
    volumeMounts:
    - name: shared-data
      mountPath: /usr/share/nginx/html
  - name: busybox-init-container
    image: busybox
    command: ["wget", "-O", "/work-dir/index.html", "http://neverssl.com/online"]
    volumeMounts:
    - name: shared-data
      mountPath: /work-dir
  volumes:
  - name: shared-data
    emptyDir: {}
In this manifest:

An NGINX container and a BusyBox init container are defined.
The NGINX container is exposed on port 80.
Both containers mount an emptyDir volume named shared-data.
The init container downloads a webpage from http://neverssl.com/online and saves it as index.html in the shared volume.
The NGINX container serves content from the same shared volume.
After applying this manifest, you can retrieve the IP address of the created pod using the kubectl get pod command:

bash
Copy code
kubectl get pod nginx-busybox-pod -o=jsonpath='{.status.podIP}'
Once you have the IP address, you can create a BusyBox pod and run wget to fetch the content served by NGINX:

bash
Copy code
kubectl run busybox --image=busybox --rm --restart=Never -- wget -O- <POD_IP>
Replace <POD_IP> with the IP address obtained from the previous command. This command will fetch the 
webpage served by NGINX in the specified pod.

11. Create a role the will allow users to get, watch, and list pods and container logs



In [ ]:

To create a role that allows users to get, watch, and list pods as well as container logs, you need 
to define a Role and a RoleBinding in Kubernetes. Below is an example YAML manifest for achieving this:

yaml
Copy code
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  namespace: <namespace>  # Replace <namespace> with the appropriate namespace
  name: pod-logs-role
rules:
- apiGroups: [""]
  resources: ["pods", "pods/log"]
  verbs: ["get", "watch", "list"]
This Role (pod-logs-role) allows users to perform get, watch, and list actions on both pods and pod logs within the specified namespace.

Next, you need to bind this role to the desired user or group. Here's an example RoleBinding:

yaml
Copy code
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: pod-logs-role-binding
  namespace: <namespace>  # Replace <namespace> with the appropriate namespace
subjects:
- kind: User
  name: <username>  # Replace <username> with the username you want to grant the role to
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: pod-logs-role
  apiGroup: rbac.authorization.k8s.io
In this RoleBinding (pod-logs-role-binding), replace <username> with the name of the user or entity 
you want to grant permissions to. This binding associates the pod-logs-role with the specified user 
within the same namespace.
After applying these manifests, the user specified in the RoleBinding will have permissions to get, 
watch, and list pods and container logs within the designated namespace. Remember to replace <namespace>
 and <username> with your specific values.





12. Create a Pod
a. name red using nginx image
b. Update the pod red to use an initContainer that uses the busybox image and sleeps
for 20 seconds

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: red
spec:
  containers:
  - name: nginx-container
    image: nginx
    ports:
    - containerPort: 80
  initContainers:
  - name: init-busybox
    image: busybox
    command: ["sh", "-c", "sleep 20"]
In this YAML:

A pod named "red" is created with an NGINX container.
An initContainer named "init-busybox" is added to the pod, which uses the BusyBox image and 
sleeps for 20 seconds before starting the main container.